# Tumor image segmentation analysis

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ../

In [ ]:
import itertools
import json
import os
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

sns.set_style('darkgrid')

savedir = 'analysis/plots'
os.makedirs(savedir, exist_ok=True)

## Post-hoc CRC only

In [ ]:
df_crc = pd.read_csv('out/polyps/crc.csv')
df_crc.head()

Plot FNR, FPR, and λ as a function of α for the post-hoc CRC method.

In [ ]:
df_crc_melt = pd.melt(df_crc, id_vars=('seed', 'alpha'), value_vars=('fnr', 'fpr', 'lambda'))
df_crc_melt = df_crc_melt.rename({'variable': 'metric'}, axis=1)

fig = plt.figure(figsize=(12, 4), tight_layout=True)

(
    so.Plot(df_crc_melt, x='alpha', y='value')
    .add(so.Dot(), so.Dodge(), so.Jitter(.3))
    .facet('metric')
    .scale(x=so.Continuous().tick(at=df_crc_melt['alpha'].unique().tolist()))
    .scale(color=so.Nominal())
    .share(y=False)
    .on(fig).plot()
)

axs = fig.get_axes()
axs[2].set(yscale='log')

## Conformal Risk Training

In [ ]:
alphas = [0.01, 0.05, 0.1]
seeds = range(10)
lrs = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
cols = ('seed', 'alpha', 'lr', 'test_fpr', 'test_fnr', 'best_epoch', 'val_fpr', 'val_lam', 'ckpt_path')

rows = []
for a, s, lr in itertools.product(alphas, seeds, lrs):
    basename = f'a{a:.2f}_lr{lr:.2g}_s{s}'
    try:
        with open(f'out/polyps/e2ecrc/{basename}.json') as f:
            result = json.load(f)
        assert result['seed'] == s
        assert result['alpha'] == a
        assert result['lr'] == lr
        result['ckpt_path'] = f'out/polyps/e2ecrc/{basename}.pth'
        rows.append({k: result[k] for k in cols})
    except FileNotFoundError:
        print(f'File not found: {basename}.json')
        continue

df_e2e = pd.DataFrame(rows)
df_e2e.head()

Plot FNR, FPR, and λ as a function of α for Conformal Risk Training. Color by learning rate

In [ ]:
df_e2e_melt = pd.melt(df_e2e, id_vars=('seed', 'alpha', 'lr'), value_vars=('test_fnr', 'test_fpr', 'val_lam'))
df_e2e_melt = df_e2e_melt.rename({'variable': 'metric'}, axis=1)

fig = plt.figure(figsize=(12, 4), tight_layout=True)

(
    so.Plot(df_e2e_melt, x='alpha', y='value', color='lr')
    .add(so.Dot(), so.Dodge(), so.Jitter(.3))
    .facet('metric')
    .scale(x=so.Continuous().tick(at=df_e2e_melt['alpha'].unique().tolist()))
    .scale(color=so.Nominal())
    .share(y=False)
    .on(fig).plot()
)

axs = fig.get_axes()
axs[2].set(yscale='log')

In [ ]:
# output columns:
# seed, alpha, lambda, fnr, fpr
best_hps = df_e2e.groupby(['alpha', 'seed'])['val_fpr'].idxmin().values.tolist()
df_e2e_best = df_e2e.loc[best_hps].reset_index().set_index(['alpha', 'seed'])

df_e2e_best = df_e2e_best.rename(columns={'test_fpr': 'fpr', 'test_fnr': 'fnr', 'val_lam': 'lambda'})
df_e2e_best = df_e2e_best.reset_index()[['seed', 'alpha', 'lambda', 'fnr', 'fpr', 'ckpt_path']]

df_e2e_best.sort_values(['seed', 'alpha']).head()

## Cross-entropy

Training pre-trained model with more epochs on cross-entropy loss.

In [ ]:
alphas = [0.01, 0.05, 0.1]
seeds = range(10)
lrs = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

rows = []
for s, lr in itertools.product(seeds, lrs):
    basename = f'lr{lr:.2g}_s{s}'
    try:
        with open(f'out/polyps/trainbase/{basename}.json') as f:
            result = json.load(f)
        assert result['seed'] == s
        assert result['lr'] == lr
        assert set(alphas).issubset(result['alphas'])
        for i, a in enumerate(result['alphas']):
            rows.append({
                'alpha': a,
                'seed': s,
                'lr': lr,
                'val_loss': result['val_loss'],
                'val_lam': result['val_lams'][i],
                'test_fpr': result['test_fprs'][i],
                'test_fnr': result['test_fnrs'][i],
                'ckpt_path': f'out/polyps/trainbase/{basename}.pt'
            })

    except FileNotFoundError:
        print(f'File not found: {basename}.json')
        continue

df_trainbase = pd.DataFrame(rows)
df_trainbase.head()

Plot FNR, FPR, and λ as a function of α for "cross-entropy" method. Color by learning rate

In [ ]:
df_trainbase_melt = pd.melt(df_trainbase, id_vars=('seed', 'alpha', 'lr'), value_vars=('test_fnr', 'test_fpr', 'val_lam'))
df_trainbase_melt = df_trainbase_melt.rename({'variable': 'metric'}, axis=1)

fig = plt.figure(figsize=(12, 4), tight_layout=True)

(
    so.Plot(df_trainbase_melt, x='alpha', y='value', color='lr')
    .add(so.Dot(), so.Dodge(), so.Jitter(.3))
    .facet('metric')
    .scale(x=so.Continuous().tick(at=df_trainbase_melt['alpha'].unique().tolist()))
    .scale(color=so.Nominal())
    .share(y=False)
    .on(fig).plot()
)

axs = fig.get_axes()
axs[2].set(yscale='log')

In [ ]:
# output columns:
# seed, alpha, lambda, fnr, fpr
best_hps = df_trainbase.groupby(['alpha', 'seed'])['val_loss'].idxmin().values.tolist()
df_trainbase_best = df_trainbase.loc[best_hps].reset_index().set_index(['alpha', 'seed'])

df_trainbase_best = df_trainbase_best.rename(columns={'test_fpr': 'fpr', 'test_fnr': 'fnr', 'val_lam': 'lambda'})
df_trainbase_best = df_trainbase_best.reset_index()[['seed', 'alpha', 'lambda', 'fnr', 'fpr', 'ckpt_path']]

df_trainbase_best.sort_values(['seed', 'alpha']).head()

## Combine

In [ ]:
df_crc['model'] = 'post-hoc CRC'
df_trainbase_best['model'] = 'cross-entropy'
df_e2e_best['model'] = 'conformal risk training'
df = pd.concat([df_crc, df_e2e_best, df_trainbase_best], axis=0)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

In [ ]:
# calculate relative FPR reduction of conformal risk training vs best baseline,
# where best baseline is min(cross-entropy, post-hoc CRC)
fprs = df.set_index(['seed', 'alpha', 'model']).unstack('model')['fpr']
fprs = fprs.loc[fprs['conformal risk training'].notna()]

fprs['best baseline'] = np.minimum(fprs['cross-entropy'], fprs['post-hoc CRC'])

rel_fpr = (fprs['best baseline'] - fprs['conformal risk training']) / fprs['best baseline']
rel_fpr.groupby('alpha').mean()

In [ ]:
df = pd.melt(df, id_vars=('seed', 'alpha', 'model'), value_vars=('fnr', 'fpr', 'lambda'))
df = df.rename({'variable': 'metric'}, axis=1)

df.head()

In [ ]:
df.groupby(['alpha', 'metric', 'model'])['value'].agg(['mean', 'std'])

In [ ]:
g = sns.catplot(
    data=df[df['alpha'] <= 0.1], x='alpha', y='value',
    hue='model', hue_order=('post-hoc CRC', 'cross-entropy', 'conformal risk training'),
    col='metric', kind='box', sharey=False,
    height=2.85
)

axes = g.axes.flatten()

axes[2].set(yscale='log')

# Iterate through the axes and add vertical lines
for ax in axes:
    ax.set(xlabel=r'$\alpha$')
    for i, label in enumerate(ax.get_xticklabels()):
        if i > 0:
            ax.axvline(i - 0.5, color="gray", linestyle="--", linewidth=1)

axes[0].set(ylabel=None, yticks=[0, 0.05, 0.1])
axes[0].set(title='False Negative Rate')
axes[1].set(title='False Positive Rate')
axes[2].set(title=r'$\lambda$')

g.tight_layout(pad=0, w_pad=1.08)
g.figure.savefig(os.path.join(savedir, 'polyps.pdf'), pad_inches=0)
g.figure.savefig(os.path.join(savedir, 'polyps.png'), pad_inches=0, dpi=300)

In [ ]:
g = sns.catplot(
    data=df[(df['alpha'] <= 0.1) & (df['metric'] != 'lambda')],
    x='alpha', y='value',
    hue='model', hue_order=('post-hoc CRC', 'cross-entropy', 'conformal risk training'),
    col='metric', kind='box', sharey=False,
    height=2.85
)

axes = g.axes.flatten()

# Iterate through the axes and add vertical lines
for ax in axes:
    ax.set(xlabel=r'$\alpha$')
    for i, label in enumerate(ax.get_xticklabels()):
        if i > 0:
            ax.axvline(i - 0.5, color="gray", linestyle="--", linewidth=1)

axes[0].set(ylabel=None, yticks=[0, 0.05, 0.1])
axes[0].set(title='False Negative Rate')
axes[1].set(title='False Positive Rate')

g.tight_layout(pad=0, w_pad=1.08)
# g.figure.savefig(os.path.join(savedir, 'polyps.pdf'), pad_inches=0)
# g.figure.savefig(os.path.join(savedir, 'polyps.png'), pad_inches=0, dpi=300)

In [ ]:
# lambda plot
fig, ax = plt.subplots(figsize=(6, 2.8), tight_layout=True)
sns.boxplot(
    data=df[(df['alpha'] <= 0.1) & (df['metric'] == 'lambda')],
    x='alpha', y='value', hue='model',
    hue_order=('post-hoc CRC', 'cross-entropy', 'conformal risk training'),
)

# place legend outside the plot on the right-hand side, vertically centered
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='model', facecolor='white', edgecolor='white')
ax.set(xlabel=r'$\alpha$', ylabel=r'$\lambda$', yscale='log')

# Iterate through the axes and add vertical lines
for i, label in enumerate(ax.get_xticklabels()):
    if i > 0:
        ax.axvline(i - 0.5, color="gray", linestyle="--", linewidth=1)

# fig.savefig(os.path.join(savedir, 'polyps_lambda.pdf'), pad_inches=0, bbox_inches='tight')
# fig.savefig(os.path.join(savedir, 'polyps_lambda.png'), pad_inches=0, bbox_inches='tight', dpi=300)

In [ ]:
# Same plot, but using seaborn objects. Unfortunately, Seaborn objects does not support boxplots.

fig = plt.figure(figsize=(12, 4), tight_layout=True)

(
    so.Plot(df[df['alpha'] <= 0.1], x="alpha", y="value", color='model')
    .add(so.Dot(), so.Dodge(), so.Jitter(.3))
    .scale(x=so.Continuous().tick(at=df['alpha'].unique().tolist()))
    .facet('metric')
    .share(y=False)
    .on(fig).plot()
)

axs = fig.get_axes()
axs[2].set(yscale='log')

# Images

Based on the best learning rates above, run

```bash
python run_polyp.py savepreds -s <seed> --tag <tag> --ckpt-path <path/to/ckpt.pt> --device cuda
```

to generate PNG images of predictions on the test set. See the README for examples.

Here, we only use seed 0 for generating images.

In [ ]:
import random
from pprint import pprint

import PIL.Image
import torch
import torchvision.transforms as transforms

IMG_SIZE = 352

In [ ]:
random.seed(5)
all_test_img_names = sorted(os.listdir('out/polyps/preds_pretrained_s0'))
test_img_names = sorted(random.sample(all_test_img_names, k=10))
test_img_names = [os.path.splitext(n)[0] for n in test_img_names]
pprint(test_img_names)

test_img_names = [
    'CVC-ClinicDB_400',
    'CVC-ColonDB_101',
    'CVC-ColonDB_291',
    'CVC-ColonDB_296',
    'CVC-ColonDB_46',
    'ETIS-LaribPolypDB_171',
    'ETIS-LaribPolypDB_43',
    'Kvasir-SEG_cju45v0pungu40871acnwtmu5',
    'Kvasir-SEG_cju88cddensj00987788yotmg',
    'Kvasir-SEG_cju88vx2uoocy075531lc63n3'
]

In [ ]:
lambdas_sr = df[(df['seed'] == 0) & (df['metric'] == 'lambda')].set_index(['alpha', 'model'])['value']
lambdas_sr

In [ ]:
gt_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE), interpolation=transforms.InterpolationMode.NEAREST),
    transforms.PILToTensor()
])

img_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])


def fn_fp_to_color(gt: torch.Tensor, pred: torch.Tensor) -> torch.Tensor:
    """
    Convert ground truth and prediction masks to a color image highlighting
    false negatives and false positives.

    Args:
        gt: ground truth mask, shape [H, W], dtype torch.bool
        pred: predicted mask, shape [H, W], dtype torch.bool
    """
    assert gt.shape == pred.shape
    assert gt.dtype == torch.bool
    assert pred.dtype == torch.bool

    h, w = gt.shape
    out = torch.zeros((3, h, w), dtype=torch.uint8)

    # correct: white or black
    out[:, (gt == 1) & (pred == 1)] = 255

    # false negative: red
    out[0, (gt == 1) & (pred == 0)] = 255

    # false positive: teal
    out[1:, (gt == 0) & (pred == 1)] = 128

    return out


def process_img(img_name: str):
    img_tensors: dict[Any, torch.Tensor] = {}

    ds, img_id = img_name.split('_')
    img_path = os.path.join('polyps/data/test', ds, 'images', f'{img_id}.png')
    gt_path = os.path.join('polyps/data/test', ds, 'masks', f'{img_id}.png')

    pil_image = PIL.Image.open(img_path)
    pil_gt = PIL.Image.open(gt_path)
    assert pil_image.mode == 'RGB'
    assert pil_gt.mode == '1'

    image = img_transform(pil_image)  # [3, H, W], torch.float32
    gt = gt_transform(pil_gt)  # [1, H, W], torch.bool
    assert isinstance(image, torch.Tensor)
    assert isinstance(gt, torch.Tensor)

    img_tensors['image'] = image
    img_tensors['gt'] = gt

    # close file handles
    pil_image.close()
    pil_gt.close()

    # pretrained predictions
    pred_path = os.path.join('out/polyps/preds_pretrained_s0_raw', f'{img_name}.npy')
    pretrained_pred_raw = torch.from_numpy(np.load(pred_path)[0])  # [H, W], torch.float32

    # cross-entropy predictions
    pred_path = os.path.join('out/polyps/preds_trainbase_s0_raw', f'{img_name}.npy')
    trainbase_pred_raw = torch.from_numpy(np.load(pred_path)[0])  # [H, W], torch.float32

    for alpha in [0.01, 0.05, 0.1]:
        lam = lambdas_sr.loc[(alpha, 'post-hoc CRC')]
        pretrained_pred = (pretrained_pred_raw >= lam)

        lam = lambdas_sr.loc[(alpha, 'cross-entropy')]
        trainbase_pred = (trainbase_pred_raw >= lam)

        lam = lambdas_sr.loc[(alpha, 'conformal risk training')]
        pred_path = os.path.join(f'out/polyps/preds_e2ecrc_a{alpha:.2f}_s0_raw', f'{img_name}.npy')
        e2e_pred = (torch.from_numpy(np.load(pred_path)[0]) >= lam)

        img_tensors[(alpha, 'pretrained')] = fn_fp_to_color(gt[0], pretrained_pred)
        img_tensors[(alpha, 'trainbase')] = fn_fp_to_color(gt[0], trainbase_pred)
        img_tensors[(alpha, 'e2e')] = fn_fp_to_color(gt[0], e2e_pred)

    alpha = 0.20
    lam = lambdas_sr.loc[(alpha, 'post-hoc CRC')]
    pretrained_pred = (pretrained_pred_raw >= lam)
    img_tensors[(alpha, 'pretrained')] = fn_fp_to_color(gt[0], pretrained_pred)

    return img_tensors

In [ ]:
# plot everything in 1 figure

num_images = 8  # len(test_img_names)
spacing = 0.05
fig, axs = plt.subplots(10, num_images, figsize=((1+spacing) * num_images + 1.5, (1+spacing)*10), gridspec_kw=dict(wspace=0, hspace=0), tight_layout=True)

for c, img_name in enumerate(test_img_names[:num_images]):
    img_tensors = process_img(img_name)

    # plot image
    ax = axs[0, c]
    ax.imshow(img_tensors['image'].permute(1, 2, 0).numpy())

    for r, alpha in enumerate([0.01, 0.05, 0.1]):
        ax = axs[3*r + 1, c]
        ax.imshow(img_tensors[(alpha, 'pretrained')].permute(1, 2, 0).numpy())

        ax = axs[3*r + 2, c]
        ax.imshow(img_tensors[(alpha, 'trainbase')].permute(1, 2, 0).numpy())

        ax = axs[3*r + 3, c]
        ax.imshow(img_tensors[(alpha, 'e2e')].permute(1, 2, 0).numpy())

    # plot alpha=0.2 pretrained
    # ax = axs[-1, c]
    # ax.imshow(img_tensors[(0.2, 'pretrained')].permute(1, 2, 0).numpy())

for ax in axs.flatten():
    ax.set(xticks=[], yticks=[])

axs[0, 0].set_ylabel('image')
axs[1, 0].set_ylabel(r'$\alpha=0.01$,' '\npost-hoc CRC')
axs[2, 0].set_ylabel(r'$\alpha=0.01$,' '\ncross-entropy')
axs[3, 0].set_ylabel(r'$\alpha=0.01$,' '\nconformal risk training')
axs[4, 0].set_ylabel(r'$\alpha=0.05$,' '\npost-hoc CRC')
axs[5, 0].set_ylabel(r'$\alpha=0.05$,' '\ncross-entropy')
axs[6, 0].set_ylabel(r'$\alpha=0.05$,' '\nconformal risk training')
axs[7, 0].set_ylabel(r'$\alpha=0.10$,' '\npost-hoc CRC')
axs[8, 0].set_ylabel(r'$\alpha=0.10$,' '\ncross-entropy')
axs[9, 0].set_ylabel(r'$\alpha=0.10$,' '\nconformal risk training')

for ax in axs[:, 0]:
    ax.yaxis.label.set(rotation='horizontal', ha='right', va='center')

fig.savefig(os.path.join(savedir, 'polyps_predictions_allinone.png'), dpi=300, bbox_inches='tight', pad_inches=0)
fig.savefig(os.path.join(savedir, 'polyps_predictions_allinone.pdf'), dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# split into 4 separate figures

num_images = 8  # len(test_img_names)
spacing = 0.05

# input images
figs = {}
axs = {}
figs['input'], axs['input'] = plt.subplots(1, num_images, figsize=((1+spacing) * num_images, 1+spacing))
for alpha in [0.01, 0.05, 0.1]:
    figs[alpha], axs[alpha] = plt.subplots(3, num_images, figsize=((1+spacing) * num_images, (1+spacing)*3))


for c, img_name in enumerate(test_img_names[:num_images]):
    img_tensors = process_img(img_name)

    # plot image
    ax = axs['input'][c]
    ax.imshow(img_tensors['image'].permute(1, 2, 0).numpy())

    for r, alpha in enumerate([0.01, 0.05, 0.1]):
        ax = axs[alpha][0, c]
        ax.imshow(img_tensors[(alpha, 'pretrained')].permute(1, 2, 0).numpy())

        ax = axs[alpha][1, c]
        ax.imshow(img_tensors[(alpha, 'trainbase')].permute(1, 2, 0).numpy())

        ax = axs[alpha][2, c]
        ax.imshow(img_tensors[(alpha, 'e2e')].permute(1, 2, 0).numpy())

    # plot alpha=0.2 pretrained
    # ax = axs[-1, c]
    # ax.imshow(img_tensors[(0.2, 'pretrained')].permute(1, 2, 0).numpy())

for ax_dict in axs.values():
    for ax in ax_dict.flatten():
        ax.axis('off')

for fig in figs.values():
    fig.tight_layout(pad=0, w_pad=0.5, h_pad=0.5)

figs['input'].savefig(os.path.join(savedir, 'polyps_predictions_input.png'), dpi=300, bbox_inches='tight', pad_inches=0)
figs['input'].savefig(os.path.join(savedir, 'polyps_predictions_input.pdf'), dpi=300, bbox_inches='tight', pad_inches=0)

for alpha in [0.01, 0.05, 0.1]:
    figs[alpha].savefig(os.path.join(savedir, f'polyps_predictions_a{alpha:.2f}.png'), dpi=300, bbox_inches='tight', pad_inches=0)
    figs[alpha].savefig(os.path.join(savedir, f'polyps_predictions_a{alpha:.2f}.pdf'), dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# split into 10 separate figures

num_images = 8  # len(test_img_names)
spacing = 0.05

# input images
figs = {}
axs = {}
figs['input'], axs['input'] = plt.subplots(1, num_images, figsize=((1+spacing) * num_images, 1+spacing))
for alpha in [0.01, 0.05, 0.1]:
    for model in ['pretrained', 'trainbase', 'e2e']:
        figs[(alpha, model)], axs[(alpha, model)] = plt.subplots(1, num_images, figsize=((1+spacing) * num_images, 1+spacing))


for c, img_name in enumerate(test_img_names[:num_images]):
    img_tensors = process_img(img_name)

    # plot image
    ax = axs['input'][c]
    ax.imshow(img_tensors['image'].permute(1, 2, 0).numpy())

    for alpha in [0.01, 0.05, 0.1]:
        for model in ['pretrained', 'trainbase', 'e2e']:
            ax = axs[(alpha, model)][c]
            ax.imshow(img_tensors[(alpha, model)].permute(1, 2, 0).numpy())

for ax_dict in axs.values():
    for ax in ax_dict.flatten():
        ax.axis('off')

for fig in figs.values():
    fig.tight_layout(pad=0, w_pad=0.5, h_pad=0.5)

figs['input'].savefig(os.path.join(savedir, 'polyps_predictions_input.png'), dpi=300, bbox_inches='tight', pad_inches=0)
figs['input'].savefig(os.path.join(savedir, 'polyps_predictions_input.pdf'), dpi=300, bbox_inches='tight', pad_inches=0)

for alpha in [0.01, 0.05, 0.1]:
    for model in ['pretrained', 'trainbase', 'e2e']:
        figs[(alpha, model)].savefig(os.path.join(savedir, f'polyps_predictions_a{alpha:.2f}_{model}.png'), dpi=300, bbox_inches='tight', pad_inches=0)
        figs[(alpha, model)].savefig(os.path.join(savedir, f'polyps_predictions_a{alpha:.2f}_{model}.pdf'), dpi=300, bbox_inches='tight', pad_inches=0)